<a href="https://colab.research.google.com/github/hyunyongPark/Recommendation/blob/master/pytorch_recom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**users**  : UserID::Gender::Age::Occupation::Zip-code

**movies**  :  MovieID::Title::Genres

**ratings** : UserID::MovieID::Rating::Timestamp



In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [3]:
# Importing the libraries
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable
# Importing the dataset
print('Importing Dataset =====>')
movies = pd.read_csv('/content/gdrive/My Drive/recommend_practice/ml-1m/movies.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
users = pd.read_csv('/content/gdrive/My Drive/recommend_practice/ml-1m/users.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
ratings = pd.read_csv('/content/gdrive/My Drive/recommend_practice/ml-1m/ratings.dat', sep = '::', header = None, engine = 'python', encoding = 'latin-1')
# Preparing the training set and the test set
print('Reading testing and training datasets =====>')
# Training and test set for 100k users
training_set_df = pd.read_csv('/content/gdrive/My Drive/recommend_practice/ml-100k/u1.base', delimiter = '\t', header=None)
test_set_df = pd.read_csv('/content/gdrive/My Drive/recommend_practice/ml-100k/u1.test', delimiter = '\t', header=None)

Importing Dataset =====>
Reading testing and training datasets =====>


In [30]:
ratings

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [32]:
users

,0,1,2,3,4,female_user,male_user
0,1,F,1,10,48067,1,0
1,2,M,56,16,70072,0,1
2,3,M,25,15,55117,0,1
3,4,M,45,7,02460,0,1
4,5,M,25,20,55455,0,1
...,...,...,...,...,...,...,...
6035,6036,F,25,15,32603,1,0
6036,6037,F,45,1,76006,1,0
6037,6038,F,56,1,14706,1,0
6038,6039,F,45,0,01060,1,0


In [4]:
training_set_df

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
...,...,...,...,...
79995,943,1067,2,875501756
79996,943,1074,4,888640250
79997,943,1188,3,888640250
79998,943,1228,3,888640275


In [5]:
# Convert training set and test set in numpy arrays
training_set_ar = np.array(training_set_df, dtype = 'int')
test_set_ar = np.array(test_set_df, dtype = 'int')
# Getting the number of users and movies
nb_users = int(max(max(training_set_ar[:,0]), max(test_set_ar[:,0])))
nb_movies = int(max(max(training_set_ar[:,1]), max(test_set_ar[:,1])))
nb_userAttributes = 4

In [6]:
print(nb_users, '\t' , nb_movies)

943 	 1682


In [7]:
users['female_user'] = (users[1] == 'F').astype(int)
users['male_user'] = (users[1] == 'M').astype(int)

In [8]:
# extract unique genre values
print('Extracting unique genres =====>')
genre = movies[2]
unique_genre = genre.unique()
genre_values = []
for movie_genre in unique_genre:
    mg = movie_genre.split("|")
    for g in mg:
        if g not in genre_values:
            genre_values.append(g)
            
genre_values = sorted(genre_values, key=str.lower)
print(genre_values)
print(len(genre_values))

Extracting unique genres =====>
['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
18


In [9]:
# get genre vector(원핫인코딩으로 이루어진 장르)
def get_genre_vector(genre_row_val):
    mg = genre_row_val.split("|")
    gen_vec = np.zeros(len(genre_values))
    gen_index = 0
    for g in genre_values:
        if g in mg:
            gen_vec[gen_index] = 1
        gen_index += 1
    return gen_vec
# unit tests for above function
print(get_genre_vector("Action|Adventure|Romance"))
print(get_genre_vector("Animation|Children's|Comedy"))
print(get_genre_vector("Thriller"))
print(get_genre_vector("Animation|Children's|Comedy|Romance"))

[1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
[0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
[0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [10]:
# Add Genre Vector to movies dataframe(위에서 한 작업을 movies데이터셋에 추가하는 작업)
print('Creating Genre vector on movies df ====>')
movie_data = movies[2]
movie_col = []
gen_index = 0
for movie_gen in movie_data:
    gen_vec = get_genre_vector(movie_gen) # 원핫인코딩 장르벡터 
    movie_col.append(gen_vec) # 그걸 movie_col에 추가해서 'genre_vector'열 추가 
    gen_index += 1
    
movies['genre_vector'] = movie_col
movies['genre_vector'][0]

Creating Genre vector on movies df ====>


array([0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.])

In [11]:
def addgenrevector(data):
    genre_array = []
    movie_id_list = data[1].tolist()  # 영화ID 리스트
    for movie_id in movie_id_list:
        genre_array.append(movies.loc[movies[0] == movie_id]['genre_vector'])  # movies데이터에서  movies영화ID와 train또는 test의 영화ID와 겹치는 부분의 장르벡터만 추출
    data['genre_vector'] = genre_array
    return data
        
print('Adding Genre Vector to training and testing datasets =====>')
training_set_gen_df = addgenrevector(training_set_df)
training_set_gen_ar = np.array(training_set_gen_df)
test_set_gen_df = addgenrevector(test_set_df)
test_set_gen_ar = np.array(test_set_gen_df)

Adding Genre Vector to training and testing datasets =====>


In [27]:
training_set_gen_df.genre_vector   # movie_id와 장르벡터의 값을 가지고 있음. (object)

0        0    [0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, ...
1        1    [0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...
2        2    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
3        3    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ...
4        4    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...
                               ...                        
79995    1053    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0....
79996        Series([], Name: genre_vector, dtype: object)
79997    1171    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0....
79998    1210    [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1....
79999    1310    [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0....
Name: genre_vector, Length: 80000, dtype: object

In [12]:
user_genre_sum = np.zeros(len(genre_values))
user_genre_sum.shape

(18,)

In [13]:
def createmultidimensionalmatrix(data):
    print(data.shape)
    gen_data = []
    
    for id_users in range(1, nb_users + 1):
        id_movies = data[1][data[0] == id_users] #nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 영화 ID values
        id_ratings = data[2][data[0] == id_users] #nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 rating values
        user_genre_list = data['genre_vector'][data[0] == id_users][data[2] >= 3] #nb_users(943)명과 train및test의 userID와 매칭되고, rating이 3이상인 train및test의 장르벡터 values
        female_user = float(users['female_user'][users[0] == id_users]) #nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 여성유저 values
        male_user = float(users['male_user'][users[0] == id_users]) #nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 남성유저 values
        user_age = float(users[2][users[0] == id_users]) #nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 나이 values
        reg_months = float(users[3][users[0] == id_users]) # nb_users(943)명과 train및test의 userID와 매칭되는 train및test의 직업 values
        user_genre_sum = np.zeros(len(genre_values)) #unique한 장르list의 길이를 갖는 zero행렬(길이=18)
        
        for usr_gen_vec in user_genre_list: # 3이상의 장르벡터의 값 
            if len(usr_gen_vec): # 길이를 지니고 있으면 T /   or F 
                user_genre_sum = user_genre_sum + np.array(usr_gen_vec) # 18개의 zero장르행렬에 평가3이상의 장르벡터를 더함. 
                ##여기까지 예를 들면 user1의 3이상의 장르벡터만을 가지는 새로운 장르벡터행렬이 생성. len(user_genre_list)*18 차원을 갖는 행렬

        data_reshaped = np.zeros(nb_movies) # 1682길이 zero행렬 (1682,)
        # 유저-영화 matrix생성
        data_reshaped[id_movies - 1] = id_ratings
        # 좋은평가만 가진 유저장르의 열 추가
        if user_genre_sum[0].shape: # (18,)
            data_reshaped = np.append(data_reshaped, user_genre_sum[0])
        else:
            data_reshaped = np.append(data_reshaped, user_genre_sum)
        # data_reshaped.shape  ->  (1700, )
        data_reshaped = np.append(data_reshaped, [female_user])
        data_reshaped = np.append(data_reshaped, [male_user])
        data_reshaped = np.append(data_reshaped, [user_age])
        data_reshaped = np.append(data_reshaped, [reg_months])
        gen_data.append(list(data_reshaped))
    return gen_data
        
        
print('Creating 2D matrix ======>')    
training_gen_data = createmultidimensionalmatrix(training_set_gen_df)
test_gen_data = createmultidimensionalmatrix(test_set_gen_df)

Creating 2D matrix ======>
(80000, 5)
(20000, 5)


In [14]:
# Converting the data into Torch tensors
print('Creating torch tensors ======>')
training_set_1 = torch.FloatTensor(training_gen_data)
test_set_1 = torch.FloatTensor(test_gen_data)

Creating torch tensors ======>


In [52]:
training_set_1

tensor([[ 5.,  3.,  4.,  ...,  0.,  1., 10.],
        [ 4.,  0.,  0.,  ...,  1., 56., 16.],
        [ 0.,  0.,  0.,  ...,  1., 25., 15.],
        ...,
        [ 5.,  0.,  0.,  ...,  1., 18., 20.],
        [ 0.,  0.,  0.,  ...,  0., 50.,  6.],
        [ 0.,  5.,  0.,  ...,  0., 45., 12.]])

In [15]:
print(training_set_1.shape)
print(test_set_1.shape)

torch.Size([943, 1704])
torch.Size([943, 1704])


In [21]:
class SAE(nn.Module):
    def __init__(self, ):
        super(SAE, self).__init__()
        self.fc1 = nn.Linear(1704, 20) # 최종 user-matrix의 컬럼수
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, 20)
        self.fc4 = nn.Linear(20, 1704)
        self.activation = nn.Sigmoid()
        
    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        return x
sae = SAE()
criterion = nn.MSELoss()
optimizer = optim.RMSprop(sae.parameters(), lr=0.01, weight_decay=0.5)

In [25]:
nb_epoch = 200
for epoch in range(1, nb_epoch+1):
    train_loss = 0
    s = 0.
    for id_user in range(nb_users):
        input = Variable(training_set_1[id_user]).unsqueeze(0)
        target = input.clone()
        #Select only rating related columns to compute loss
        target_ratings = target[:, :nb_movies]
        if torch.sum(target.data > 0) > 0:
            output = sae(input)
            output_ratings = output[:, :nb_movies]
            target.require_grad = False
            output[target == 0] = 0
            loss = criterion(output_ratings, target_ratings)
            mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
            loss.backward()
            train_loss += np.sqrt(loss.item()*mean_corrector)
            s += 1.
            optimizer.step()
    print('epoch: '+str(epoch)+' loss: '+str(train_loss/s))
    
    
# Testing the SAE
test_loss = 0
s = 0.
for id_user in range(nb_users):
    input = Variable(training_set_1[id_user]).unsqueeze(0)
    target = Variable(test_set_1[id_user]).unsqueeze(0)
    target_ratings = target[:, :nb_movies]
    if torch.sum(target.data > 0) > 0:
        output = sae(input)
        output_ratings = output[:, :nb_movies]
        target.require_grad = False
        output[target == 0] = 0
        loss = criterion(output_ratings, target_ratings)
        mean_corrector = nb_movies/float(torch.sum(target.data > 0) + 1e-10)
        test_loss += np.sqrt(loss.item()*mean_corrector)
        s += 1.
print('test loss: '+str(test_loss/s))

epoch: 1 loss: 0.8679231755874817
epoch: 2 loss: 0.8671374799601206
epoch: 3 loss: 0.8676912617591324
epoch: 4 loss: 0.8678975911366743
epoch: 5 loss: 0.8677578057097864
epoch: 6 loss: 0.8676387097714362
epoch: 7 loss: 0.8671501817503691
epoch: 8 loss: 0.8679018927986164
epoch: 9 loss: 0.867692918705164
epoch: 10 loss: 0.8676996238935173
epoch: 11 loss: 0.8678647696765625
epoch: 12 loss: 0.867490860444076
epoch: 13 loss: 0.8674225025648874
epoch: 14 loss: 0.8673119943283719
epoch: 15 loss: 0.8675490790586149
epoch: 16 loss: 0.8671608701378248
epoch: 17 loss: 0.8670920138462047
epoch: 18 loss: 0.8669289734463664
epoch: 19 loss: 0.8672916557497173
epoch: 20 loss: 0.8670303226966065
epoch: 21 loss: 0.8670548801691307
epoch: 22 loss: 0.8672266981833835
epoch: 23 loss: 0.867787752324975
epoch: 24 loss: 0.86699003352245
epoch: 25 loss: 0.8673717379995435
epoch: 26 loss: 0.8675253850892035
epoch: 27 loss: 0.8668086683937645
epoch: 28 loss: 0.86726775721136
epoch: 29 loss: 0.8672401910017022
e

In [30]:
sae(test_set_1)

tensor([[ 3.9070e+00,  3.3209e+00,  3.1220e+00,  ...,  2.7881e-03,
          1.1244e-02, -1.4509e-02],
        [ 3.8663e+00,  3.2853e+00,  3.0878e+00,  ...,  2.7686e-03,
          1.1180e-02, -1.4319e-02],
        [ 3.8954e+00,  3.3108e+00,  3.1122e+00,  ...,  2.7825e-03,
          1.1226e-02, -1.4455e-02],
        ...,
        [ 3.5634e+00,  3.0201e+00,  2.8336e+00,  ...,  2.6470e-03,
          1.0731e-02, -1.2853e-02],
        [ 3.4266e+00,  2.9003e+00,  2.7190e+00,  ...,  2.6028e-03,
          1.0543e-02, -1.2168e-02],
        [ 3.4300e+00,  2.9033e+00,  2.7219e+00,  ...,  2.6038e-03,
          1.0548e-02, -1.2186e-02]], grad_fn=<AddmmBackward>)